In [1]:
import asyncio
import redis.asyncio as redis
from ubud.redis.db import Database

r = redis.Redis(decode_responses=True)
db = Database(redis_client=r)

In [2]:
# Redis Key 확인
await db.keys()

['ubud/exchange/balance/bithumb/BTC',
 'ubud/exchange/balance/bithumb/ETH',
 'ubud/exchange/balance/bithumb/KRW',
 'ubud/exchange/balance/bithumb/WAVES',
 'ubud/exchange/balance/ftx/BTC',
 'ubud/exchange/balance/upbit/KRW',
 'ubud/forex/FRX.KRWUSD',
 'ubud/keys',
 'ubud/quotation/keys',
 'ubud/quotation/orderbook/bithumb/BTC/KRW/ask',
 'ubud/quotation/orderbook/bithumb/BTC/KRW/bid',
 'ubud/quotation/orderbook/bithumb/ETH/KRW/ask',
 'ubud/quotation/orderbook/bithumb/ETH/KRW/bid',
 'ubud/quotation/orderbook/bithumb/WAVES/KRW/ask',
 'ubud/quotation/orderbook/bithumb/WAVES/KRW/bid',
 'ubud/quotation/orderbook/ftx/BTC/PERP/ask',
 'ubud/quotation/orderbook/ftx/BTC/PERP/bid',
 'ubud/quotation/orderbook/ftx/BTC/USD/ask',
 'ubud/quotation/orderbook/ftx/BTC/USD/bid',
 'ubud/quotation/orderbook/ftx/ETH/PERP/ask',
 'ubud/quotation/orderbook/ftx/ETH/PERP/bid',
 'ubud/quotation/orderbook/ftx/ETH/USD/ask',
 'ubud/quotation/orderbook/ftx/ETH/USD/bid',
 'ubud/quotation/orderbook/ftx/WAVES/PERP/ask',
 '

In [3]:
# Pattern으로 Redis Key 조회
await db.keys("ubud/quotation/orderbook/*/BTC/*")

['ubud/quotation/orderbook/bithumb/BTC/KRW/bid',
 'ubud/quotation/orderbook/upbit/BTC/KRW/bid',
 'ubud/quotation/orderbook/ftx/BTC/PERP/bid',
 'ubud/quotation/orderbook/upbit/BTC/KRW/ask',
 'ubud/quotation/orderbook/ftx/BTC/PERP/ask',
 'ubud/quotation/orderbook/ftx/BTC/USD/bid',
 'ubud/quotation/orderbook/bithumb/BTC/KRW/ask',
 'ubud/quotation/orderbook/ftx/BTC/USD/ask']

In [4]:
# 김프 구하기

async def get_gimp(_print=True):
    # Upbit의 BTC 가격
    x = await db.get("ubud/quotation/orderbook/upbit/BTC/KRW/bid")
    krw_btc_upbit = x["price"]

    # FTX의 BTC 가격 (원화 환산)
    x = await db.get("ubud/forex/FRX.KRWUSD")
    krw_per_usd = x["basePrice"]
    x = await db.get("ubud/quotation/orderbook/ftx/BTC/USD/bid")
    usd_btc_ftx = x["price"]
    krw_btc_ftx = usd_btc_ftx * krw_per_usd

    # 김프
    gimp = krw_btc_upbit/krw_btc_ftx
    if _print:
        print(f"UPBIT : {krw_btc_upbit:.1f} KRW/BTC")
        print(f"FTX   : {krw_btc_ftx:.1f} KRW/BTC")
        print(f"GIMP! : {gimp:.3f} BTC/BTC")
    
    return gimp

_ = await get_gimp()

UPBIT : 27446000.0 KRW/BTC
FTX   : 27183000.0 KRW/BTC
GIMP! : 1.010 BTC/BTC


In [5]:
# 설정 값 넘기는 것 10번 보고 종료
# 설정 값 안 넘으면 1,000회 중 최대 김프 반환
MIN_GIMP = 1.01

i = 0
j = 0
max_gimp = 0
while True:
    gimp = await get_gimp(_print=False)
    if gimp > MIN_GIMP:
        j += 1
        print(f"SUPER GIMP {j} TIMES! - {gimp}")
        if j > 10:
            break
    if gimp > max_gimp:
        max_gimp = gimp
    i += 1
    if j > 1000:
        print(f"NO SUPER GIMP TODAY! - max gimp is {max_gimp}")
        break
    await asyncio.sleep(0.001)

SUPER GIMP 1 TIMES! - 1.0143320693911642
SUPER GIMP 2 TIMES! - 1.0103763209486922
SUPER GIMP 3 TIMES! - 1.0103763209486922
SUPER GIMP 4 TIMES! - 1.0103763209486922
SUPER GIMP 5 TIMES! - 1.0110026834915575
SUPER GIMP 6 TIMES! - 1.0110026834915575
SUPER GIMP 7 TIMES! - 1.0115351468410934
SUPER GIMP 8 TIMES! - 1.0114983168703364
SUPER GIMP 9 TIMES! - 1.0114983168703364
SUPER GIMP 10 TIMES! - 1.0101441066344958
SUPER GIMP 11 TIMES! - 1.0101073259722968


In [6]:
# [NOTE] REDIS STREAM을 Trigger로 사용하여 데이터 업데이트 있을 때만 작동!
# 설정 값 넘기는 것 10번 보고 종료
# 설정 값 안 넘으면 1,000회 중 최대 김프 반환
MIN_GIMP = 1.01

i = 0
j = 0
max_gimp = 0
while True:
    stream = await r.xread({"ubud-stream/quotation/orderbook/ftx/BTC/USD/bid": "$"}, count=1, block=1)
    if len(stream) > 0:
        gimp = await get_gimp(_print=False)
        if gimp > MIN_GIMP:
            j += 1
            print(f"SUPER GIMP {j} TIMES! - {gimp}")
            if j > 10:
                break
        if gimp > max_gimp:
            max_gimp = gimp
        i += 1
        if j > 1000:
            print(f"NO SUPER GIMP TODAY! - max gimp is {max_gimp}")
            break
        await asyncio.sleep(0.001)

SUPER GIMP 1 TIMES! - 1.0104527051895473
SUPER GIMP 2 TIMES! - 1.0106050005523437
SUPER GIMP 3 TIMES! - 1.011327814276876
SUPER GIMP 4 TIMES! - 1.0102296612321975
SUPER GIMP 5 TIMES! - 1.0103373431924365
SUPER GIMP 6 TIMES! - 1.0102983542501407
SUPER GIMP 7 TIMES! - 1.0130387473950169
SUPER GIMP 8 TIMES! - 1.0122913186339046
SUPER GIMP 9 TIMES! - 1.0106439771877958
SUPER GIMP 10 TIMES! - 1.0113195867171725
SUPER GIMP 11 TIMES! - 1.014380804896348
